In [15]:
import pandas as pd
import re
from time import time
from pprint import pprint
from datetime import datetime
print(datetime.now())

2017-11-22 14:55:30.514929


In [9]:
from pyspark.mllib.fpm import FPGrowth

In [10]:
#config
datafile = '/home/test/Documents/DataCleansing/Transaction Classification Problem/dataset_5BCDE.csv'
processed_data_out='/home/test/Documents/DataCleansing/Transaction Classification Problem/Processed_1M.csv'
fp_tree_1M = '/home/test/Documents/DataCleansing/Transaction Classification Problem/fp_tree_1M.csv'

parts = 10
num_obs=10

In [11]:
num_rep='\d{4,}'

In [12]:
data = sc.textFile(datafile,minPartitions=parts)


header = data.first()
data = data.filter(lambda line: line != header)
# print(data.take(num_obs),'\n')
data = data.map(lambda x: x.split("|"))
# print(data.take(num_obs),'\n')
data = data.map(lambda x: x[1])
# print(data.take(num_obs),'\n')
data = data.map(lambda line:re.sub('[^A-Za-z0-9]+', ',', line))
print(data.take(num_obs),'\n')
data = data.map(lambda line:re.sub(num_rep, 'NUM', line))
print(data.take(num_obs),'\n')
data = data.map(lambda line: line.lower().strip().split(',')[:3])
print(data.take(num_obs),'\n')
data = data.map(lambda line: [elem+'_P'+str(idx+1) for (elem,idx) in zip(line,range(len(line)))])
print(data.take(num_obs),'\n')
# data=data.cache()


['TD007411000169,TDS', 'Closure,proceeds,credited,to,TD039111000395,', 'VLT,TO,T09', 'CASH,RECEIPT', 'BNG,CS,LOADED,AS,AT,02,5,16', 'CASH,RECEIPT', 'CASH,RECEIPT', 'CASH,RECEIPT', 'CASH,RECEIPT', 'VLT,T1'] 

['TDNUM,TDS', 'Closure,proceeds,credited,to,TDNUM,', 'VLT,TO,T09', 'CASH,RECEIPT', 'BNG,CS,LOADED,AS,AT,02,5,16', 'CASH,RECEIPT', 'CASH,RECEIPT', 'CASH,RECEIPT', 'CASH,RECEIPT', 'VLT,T1'] 

[['tdnum', 'tds'], ['closure', 'proceeds', 'credited'], ['vlt', 'to', 't09'], ['cash', 'receipt'], ['bng', 'cs', 'loaded'], ['cash', 'receipt'], ['cash', 'receipt'], ['cash', 'receipt'], ['cash', 'receipt'], ['vlt', 't1']] 

[['tdnum_P1', 'tds_P2'], ['closure_P1', 'proceeds_P2', 'credited_P3'], ['vlt_P1', 'to_P2', 't09_P3'], ['cash_P1', 'receipt_P2'], ['bng_P1', 'cs_P2', 'loaded_P3'], ['cash_P1', 'receipt_P2'], ['cash_P1', 'receipt_P2'], ['cash_P1', 'receipt_P2'], ['cash_P1', 'receipt_P2'], ['vlt_P1', 't1_P2']] 



In [16]:
datamodel = FPGrowth.train(data,minSupport=0.0005,numPartitions=10)


In [9]:
# data.saveAsTextFile(processed_data_out)

In [10]:
# df=data.toDF()
# data.take(100)
data.getNumPartitions()

10

In [15]:
df.head(4),df.first()

([Row(_1='trf_P1', _2='ifo_P2', _3='crsg_P3'),
  Row(_1='fee_P1', _2='on_P2', _3='od_P3'),
  Row(_1='shortfall_P1', _2='iro_P2', _3='crsg_P3'),
  Row(_1='part_P1', _2='pymt_P2', _3='num_P3')],
 Row(_1='trf_P1', _2='ifo_P2', _3='crsg_P3'))

In [17]:
result = datamodel.freqItemsets().collect()
print(len(result))
# result[:10],result[-10:]
print(datetime.now())

291
2017-11-22 15:00:51.064140


In [8]:
data_length = data.count()
data_length

30704991

In [9]:
#creating list of fp_tree
fp_tree_list =[[item[0],item[1],round(100*item[1]/data_length,3)] for item in result]
len(fp_tree_list)
pprint(fp_tree_list[:5])

[[['3qtl002_P3'], 103175, 0.336],
 [['3qtl002_P3', 'web_P1'], 103175, 0.336],
 [['3qtl002_P3', 'web_P1', 'trf_P2'], 103175, 0.336],
 [['3qtl002_P3', 'trf_P2'], 103175, 0.336],
 [['vtairtelnum_P1'], 34725, 0.113]]


In [20]:
#create dictionary of Fp tree
fp_tree_dict = {}
i=0
for elem in result:
    i+=1
    fp_tree_dict[str(elem[0])]=[elem[1],round(100*elem[1]/data_length,2)]
# result[0][1]
# print(fp_tree_dict.items())

In [10]:
#creating dataframe of FP tree
import pandas as pd
fp_df = pd.DataFrame(fp_tree_list,columns=['tree_map', 'absolute_support', 'support_ratio'])
print(fp_df.head())

                       tree_map  absolute_support  support_ratio
0                  [3qtl002_P3]            103175          0.336
1          [3qtl002_P3, web_P1]            103175          0.336
2  [3qtl002_P3, web_P1, trf_P2]            103175          0.336
3          [3qtl002_P3, trf_P2]            103175          0.336
4              [vtairtelnum_P1]             34725          0.113


In [11]:
fp_df['num_Ps']=fp_df['tree_map'].apply(lambda x:len(x))

In [12]:
fp_df.head(20)

,tree_map,absolute_support,support_ratio,num_Ps
0,[3qtl002_P3],103175,0.336,1
1,"[3qtl002_P3, web_P1]",103175,0.336,2
2,"[3qtl002_P3, web_P1, trf_P2]",103175,0.336,3
3,"[3qtl002_P3, trf_P2]",103175,0.336,2
4,[vtairtelnum_P1],34725,0.113,1
5,[tnf_P1],595496,1.939,1
6,[self_P2],150107,0.489,1
7,"[self_P2, cdb_P1]",55847,0.182,2
8,"[self_P2, cdb_P1, num_P3]",51897,0.169,3
9,"[self_P2, num_P3]",109888,0.358,2


In [65]:
fp_df.num_Ps.value_counts()

1    88
2    85
3    27
Name: num_Ps, dtype: int64

In [61]:
fp_df.groupby('num_Ps').agg('mean')

,absolute_support,support_ratio
num_Ps,,
1,924733.556818,3.011670
2,880127.352941,2.866424
3,879216.740741,2.863333


In [13]:
def create_elem(l,i):
    if len(l) > i:
        return l[i]
    else:
        return ''

In [14]:
fp_df['Key_1'] = fp_df['tree_map'].apply(lambda l:l[0])
fp_df['Key_2'] = fp_df['tree_map'].apply(create_elem,args=(1,))
fp_df['Key_3'] = fp_df['tree_map'].apply(create_elem,args=(2,))

print(len(fp_df[fp_df['Key_3'] != '']))
fp_df[fp_df['Key_3'] != '']

27


,tree_map,absolute_support,support_ratio,num_Ps,Key_1,Key_2,Key_3
2,"[3qtl002_P3, web_P1, trf_P2]",103175,0.336,3,3qtl002_P3,web_P1,trf_P2
8,"[self_P2, cdb_P1, num_P3]",51897,0.169,3,self_P2,cdb_P1,num_P3
13,"[trf_P2, atm_P1, num_P3]",1046775,3.409,3,trf_P2,atm_P1,num_P3
33,"[tr_P3, r_P1, num_P2]",101320,0.330,3,tr_P3,r_P1,num_P2
36,"[april_P3, card_P1, maint_P2]",284458,0.926,3,april_P3,card_P1,maint_P2
42,"[wd_P2, atm_P1, num_P3]",11890928,38.726,3,wd_P2,atm_P1,num_P3
48,"[adj_P3, book_P2, num_P1]",454389,1.480,3,adj_P3,book_P2,num_P1
52,"[cwb_P1, self_P2, num_P3]",40038,0.130,3,cwb_P1,self_P2,num_P3
66,"[ngn_P3, revaluation_P1, of_P2]",41197,0.134,3,ngn_P3,revaluation_P1,of_P2
82,"[book_P1, interest_P2, for_P3]",59114,0.193,3,book_P1,interest_P2,for_P3


In [15]:

fp_df.to_csv(fp_tree_1M,sep = '|')